# Script that bins and averages AAQ data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mod_aaq

## --- Configurations

### Settings to be changed for each data

- dir_name: Directory of the folder where the AAQ data is stored
- core_name: AAQ file name (without .csv)
- bottom_id: Bottom cell ID (row number) in the csv file, below which the data is cutoff from the analysis
- write_flag: Option to write output
    - write_flag = 1: yes
    - write_flag = 2: no

In [2]:
dir_name = 'C:/Users/yoshi/OneDrive/ドキュメント/Python Scripts/aaq_averaging'
core_name = 'AAQ176-CAD_0434_20221128_075426_0001'
bottom_id = 163
write_flag = 1

### Settings for averaging

- dz: Bin thickness for averaging (m)
- i_opt: Option to treat blank bins
    - i_opt = 1: remove blank bins
    - i_opt = 2: linear interpolation from the adjacent bins
    - otherwise: leave blank bins as is

In [3]:
dz = 0.10
i_opt = 1

### Settings for plotting

- fig_flag: Option to make a plot
    - fig_flag = 1: yes
    - otherwise: no
- fig_var_id: Choice of 4 variables to plot

| Temp | Sal | Cond | EC25 | Density | SigmaT | Chl-Flu | Chl-a | Turb | pH | DO (%) | DO (mg/L) | PAR | 
|:----:|:---:|:----:|:----:|:-------:|:------:|:-------:|:-----:|:----:|:--:|:------:|:---------:|:---:|
| 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 |
<style>
table {float:left}
</style>

In [4]:
fig_flag = 1
fig_var_id = [1, 2, 8, 12]

### Axis control

- xasis_adjust: Adjust x-axes of the four variables [plot1, plot2, plot3, plot4]
    - xasis_adjust = 1: yes
    - otherwise: auto-adjust
- yasis_adjust: Adjust y-axes (Depth)
    - yasis_adjust = 1: yes
    - otherwise: auto-adjust
- dep_lim: Y-axis (depth) range; [mix, max]: (when yaxis_adjust = 1)
- dep_dy: Y-axis tick interval: (when yaxis_adjust = 1)
- xlim_XX: X-axis range for each variable; [mix, max]: (when xaxis_adjust = 1)
- dx_XX: X-axis tick interval for each variable: (when yaxis_adjust = 1)

In [5]:
xaxis_adjust = [1, 1, 1, 1]
yaxis_adjust = 1
# --- Y axis range and tick interval --- #
dep_lim     = [0, 3]
dep_dy      = 1
# --- X-axis range --- #
xlim_temp    = [26.0, 31.0]
xlim_sal     = [30, 34]
xlim_cond    = [0, 100]
xlim_EC25    = [40000, 50000]
xlim_density = [1000, 1030]
xlim_sigmaT  = [0, 1030]
xlim_Chl_flu = [0, 50]
xlim_Chl_a   = [0, 50]
xlim_turb    = [0, 50]
xlim_ph      = [7, 8.5]
xlim_DO_sat  = [0, 100]
xlim_DO      = [0, 8]
xlim_PAR     = [0, 1000]
# --- X-axis tick interval --- #
dx_temp    = 2
dx_sal     = 2
dx_cond    = 20
dx_EC25    = 10000
dx_density = 2
dx_sigmaT  = 2
dx_Chl_flu = 10
dx_Chl_a   = 10
dx_turb    = 10
dx_ph      = 0.2
dx_DO_sat  = 10
dx_DO      = 2
dx_PAR     = 100

### Figure control

- fig_size: Figure size [width, height]
- fig_space: Spaces between the subplots [wspace, hspace]
- data_linethck: Data line thickness
- ax_linethck: Axis line thickness
- fontsize_ax: Axis ticks font size
- fontsize_label: Axis label font size
- marker_option: Include markers at data points; 'on': yes, otherwise: no
    - marker_option = 'on': yes
    - otherwise: no
- marker_sybol: Marker symbol; e.g. 'o', '^', 'v', 's', 'x'
- marker_size: Marker size

In [6]:
fig_size = [6.0, 7.0]
fig_space = [0.4, 0.3]
data_linethck  = 1.2
ax_linethck    = 0.7
fontsize_ax    = 9
fontsize_label = 12
marker_option  = 'on'
marker_sybol   = 'o'
marker_size    = 5

## --- End of Configurations

## &darr; Code of averaging & plotting &darr;

In [7]:
def main(dir_name, core_name, bottom_id, dz, i_opt):
    mod_aaq

if __name__ == '__main__':
    
    # Read AAQ csv file
    [headers, data_array] = mod_aaq.read_aaq_data(dir_name, core_name, \
                                                  bottom_id)
    
    # Binning data along water depth with a thickness dz    
    [dep_bin, bin_array] = mod_aaq.aaq_bin(data_array, dz, i_opt)
    
    # Write binned data to csv file
    mod_aaq.csv_write_aaq(write_flag, core_name, headers, bin_array)
    
    # Make plots
        
    plt.close('all')
    if fig_flag == 1:
        
        # Make plots in a seperate window
        %matplotlib qt
        
        fig = plt.figure(figsize=(fig_size))
        
        axes= fig.subplots(2, 2)
    
        plt.subplots_adjust(wspace=fig_space[0], hspace=fig_space[1])
        
        for i in range(0,4):
            [bin_data, x_lim, x_ticks, xlabel] = mod_aaq.fig_param(fig_var_id[i], \
                              bin_array, xlim_temp, xlim_sal, xlim_cond, 
                              xlim_EC25, xlim_density, xlim_sigmaT, xlim_Chl_flu, \
                              xlim_Chl_a, xlim_turb, xlim_ph, xlim_DO_sat, \
                              xlim_DO, xlim_PAR, dx_temp, dx_sal, dx_cond, \
                              dx_EC25, dx_density, dx_sigmaT, dx_Chl_flu, \
                              dx_Chl_a, dx_turb, dx_ph, dx_DO_sat, dx_DO, dx_PAR)
                
            if i == 0:
                ax = axes[0][0]
            elif i == 1:
                ax = axes[0][1]
            elif i == 2:
                ax = axes[1][0]
            elif i == 3:
                ax = axes[1][1]
    
            if marker_option == 'on':
                ax.plot(bin_data, dep_bin, marker=marker_sybol, linestyle='-',
                        linewidth=data_linethck, markersize=marker_size)
            else:
                ax.plot(bin_data, dep_bin, linewidth=data_linethck)
                    
            if xaxis_adjust[i] == 1:
                ax.set_xlim(x_lim[0],x_lim[1])
                ax.set_xticks(x_ticks) 
                ax.set_xticklabels(x_ticks,fontsize=fontsize_ax)
            else:
                xticklabels = ax.get_xticklabels()
                xticks = ax.get_xticks()
                ax.set_xticks(xticks) 
                ax.set_xticklabels(xticklabels,fontsize=fontsize_ax)
                
            if yaxis_adjust == 1:
                dep_ticks = np.arange(dep_lim[0], dep_lim[1]+0.5*dep_dy, dep_dy)    
                ax.set_ylim(dep_lim[0],dep_lim[1])
                ax.set_yticks(dep_ticks) 
                ax.set_yticklabels(dep_ticks,fontsize=fontsize_ax)
            else:
                yticklabels = ax.get_yticklabels()
                #yticks = ax.get_yticks()
                #ax.set_yticks(yticks) 
                ax.set_yticklabels(yticklabels,fontsize=fontsize_ax)
            
            ax.invert_yaxis()
            ax.set_xlabel(xlabel,fontsize=fontsize_label)
            ax.set_ylabel('Depth (m)',fontsize=fontsize_label)
            
            # change all spines
            for axis in ['top','bottom','left','right']:
                ax.spines[axis].set_linewidth(ax_linethck)
            # increase tick width
            ax.tick_params(width=ax_linethck)
        
        plt.show()
